In [1]:
import $ivy.`io.github.mullerhai::storch_core:0.6.1-1.15.2`

import $ivy.$                                              


import java.net.URL

import java.util.zip.GZIPInputStream

import java.nio.file.{Paths, Files, Path}


In [2]:
import org.bytedeco.pytorch
import org.bytedeco.javacpp.{FloatPointer, PointerScope}
import org.bytedeco.pytorch.{Example, InputArchive, OutputArchive, TensorExampleVectorIterator}
import org.bytedeco.pytorch.{ChunkDatasetOptions, Example, ExampleIterator, ExampleStack, ExampleVector}
import org.bytedeco.pytorch.global.torch as torchNative
import java.net.URL
import java.util.zip.GZIPInputStream
import java.nio.file.{Paths, Files, Path}
import scala.collection.{mutable, Set as KeySet}
import scala.util.{Try,Success,Failure,Using,Random}
import torch.Device.{CPU, CUDA}
import torch.internal.NativeConverters.{fromNative, toNative}
import torch.{---, ::, &&, Slice}
import torch.nn.functional as F
import torch.nn.modules.HasParams
import torch.optim.Adam
import torch.{ BFloat16 ,Float32 , Int64,Default, Tensor }
import torch.nn as nn
import torch.utils.data.{NormalTensorDataset,DataLoaderOptions,Dataset, DataLoader}
import torch.utils.data.dataset.custom.{MNIST, FashionMNIST}
import torch.utils.data.*
import torch.utils.data.dataloader.*
import torch.utils.data.datareader.ChunkDataReader
import torch.utils.data.dataset.*
import torch.utils.data.sampler.RandomSampler
import torch.numpy.TorchNumpy
import torch.numpy.enums.DType.Float32 as NPFloat32
import torch.numpy.matrix.NDArray
import TorchNumpy.loadNDArrayFromCSV
import torch.pandas.DataFrame
// import torch.Tensor.fromNative

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


import org.bytedeco.pytorch

import org.bytedeco.javacpp.{FloatPointer, PointerScope}

import org.bytedeco.pytorch.{Example, InputArchive, OutputArchive, TensorExampleVectorIterator}

import org.bytedeco.pytorch.{ChunkDatasetOptions, Example, ExampleIterator, ExampleStack, ExampleVector}

import org.bytedeco.pytorch.global.torch as torchNative

import java.net.URL

import java.util.zip.GZIPInputStream

import java.nio.file.{Paths, Files, Path}

import scala.collection.{mutable, Set as KeySet}

import scala.util.{Try,Success,Failure,Using,Random}

import torch.Device.{CPU, CUDA}

import torch.internal.NativeConverters.{fromNative, toNative}

import torch.{---, ::, &&, Slice}

import torch.nn.functional as F

import torch.nn.modules.HasParams

import torch.optim.Adam

import torch.{ BFloat16 ,Float32 , Int64,Default, Tensor }

import torch.nn as nn

import torch.utils.data.{NormalTensorDataset,DataLoaderOptions,Dataset, DataLoader}

import torch.utils.data.dataset.custom.{MNIST, FashionM

In [3]:
val device = if torch.cuda.isAvailable then CUDA else CPU
println(s"Using device: $device")
//  val model  = GruNet().to(device)
  //  val model = LstmNet().to(device)
//  val model = RnnNet().to(device)
val modelPahth ="D:\\data\\git\\storch-tutorial\\lstm-netss.pt"
val dataPath = Paths.get("D:\\data\\FashionMNIST")
val mnistTrain = FashionMNIST(dataPath, train = true, download = true)
val mnistEval = FashionMNIST(dataPath, train = false)
val trainFeatures = mnistTrain.features.to(device)
val trainTargets = mnistTrain.targets.to(device)
val evalFeatures = mnistEval.features.to(device)
val evalTargets = mnistEval.targets.to(device)
val r = Random(seed = 0)
val exampleSeq = mnistTrain.map(x => new Example(x._1.native, x._2.native))
// val lossFn = torch.nn.loss.CrossEntropyLoss()
// val model = new  RnnNet[Float32]().to(device)
// val optimizer = Adam(model.parameters, lr = 1e-3, amsgrad = true)
// val optimizerCopy = Adam(model.parameters, lr = 1e-3, amsgrad = true)

val num_epochs = 10
val criterion = torch.nn.loss.CrossEntropyLoss()

// val exampleSeq = mnistTrain.map(x => new Example(x._1.native, x._2.native))
    //  val ex1 = new Example(mnistTrain.features.native ,mnistTrain.targets.native)
val exampleVector = new ExampleVector(exampleSeq *)

Using device: Device(CPU,-1)


device: Device = Device(device = CPU, index = -1)
modelPahth: String = "D:\\data\\git\\storch-tutorial\\lstm-netss.pt"
dataPath: Path = D:\data\FashionMNIST
mnistTrain: FashionMNIST = IndexedSeq(
  (
    tensor dtype=float32, shape=[1, 28, 28], device=CPU 
[[[0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  ...,
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000]]],
    tensor dtype=int64, shape=[], device=CPU 
9
  ),
  (
    tensor dtype=float32, shape=[1, 28, 28], device=CPU 
[[[0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  ...,
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0

In [4]:
class LstmNet[D <: BFloat16 | Float32 : Default](
                                                  inputSize: Int = 28,
                                                  hiddenSize: Int = 128,
                                                  numLayers: Int = 2,
                                                  numClasses: Int = 10
                                                ) extends HasParams[D] {

  val lstm = register(nn.LSTM(inputSize, hiddenSize, numLayers, batch_first = true))
  val fc = register(nn.Linear(hiddenSize, numClasses))

  def apply(i: Tensor[D]): Tensor[D] =
    val arr = Seq(numLayers, i.size.head, hiddenSize.toInt)
    val h0 = torch.zeros(size = arr, dtype = i.dtype)
    val c0 = torch.zeros(size = arr, dtype = i.dtype)
    val outTuple3 = lstm(i, Some(h0), Some(c0))
    var out: Tensor[D] = outTuple3._1
    out = out.index(torch.indexing.::, -1, ::)
    F.logSoftmax(fc(out), dim = 1)

}
// import torch.*
class RnnNet[D <: BFloat16 | Float32 : Default](
                                                 inputSize: Int = 28,
                                                 hiddenSize: Int = 128,
                                                 numLayers: Int = 2,
                                                 numClasses: Int = 10
                                               ) extends HasParams[D] {

  val rnn = register(nn.RNN(inputSize, hiddenSize, numLayers, batch_first = true))
  val fc = register(nn.Linear(hiddenSize, numClasses))

  def apply(i: Tensor[D]): Tensor[D] =
    val arr = Seq(numLayers, i.size.head, hiddenSize.toInt)
    val h0 = torch.zeros(size = arr, dtype = i.dtype)
    val c0 = torch.zeros(size = arr, dtype = i.dtype)
    val outTuple2 = rnn(i, Some(h0))
    var out: Tensor[D] = outTuple2._1
    out = out.index(torch.indexing.::, -1, ::)
    F.logSoftmax(fc(out), dim = 1)
}

class GruNet[D <: BFloat16 | Float32 : Default](
                                                 inputSize: Int = 28,
                                                 hiddenSize: Int = 128,
                                                 numLayers: Int = 2,
                                                 numClasses: Int = 10
                                               ) extends HasParams[D] {

  val gru = register(nn.GRU(inputSize, hiddenSize, numLayers, batch_first = true))
  val fc = register(nn.Linear(hiddenSize, numClasses))

  def apply(i: Tensor[D]): Tensor[D] =
    val arr = Seq(numLayers, i.size.head, hiddenSize.toInt)
    val h0 = torch.zeros(size = arr, dtype = i.dtype)
    val c0 = torch.zeros(size = arr, dtype = i.dtype)
    val outTuple2 = gru(i, Some(h0))
    var out: Tensor[D] = outTuple2._1
    out = out.index(torch.indexing.::, -1, ::)
    F.logSoftmax(fc(out), dim = 1)
}

val model = new  RnnNet[Float32]().to(device)
val optimizer = Adam(model.parameters, lr = 1e-3, amsgrad = true)
val optimizerCopy = Adam(model.parameters, lr = 1e-3, amsgrad = true)

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


defined class LstmNet
defined class RnnNet
defined class GruNet
model: RnnNet[Float32] = RnnNet
optimizer: Adam = torch.optim.Adam[address=0x0,position=0,limit=0,capacity=0,deallocator=null]
optimizerCopy: Adam = torch.optim.Adam[address=0x0,position=0,limit=0,capacity=0,deallocator=null]

In [ ]:
val batchSize = 100
val timeout = 10.0f
val sampler = new RandomSampler(mnistTrain.length)
val batchSampler = new RandomSampler(mnistTrain.length)
val trainLoader = new DataLoader(dataset = mnistTrain, batch_size= batchSize, shuffle = true, sampler = sampler, batch_sampler= batchSampler, timeout = timeout)
val evalLoader = new DataLoader(dataset = mnistEval,batch_size= batchSize, shuffle = false, sampler = sampler, batch_sampler= batchSampler, timeout = timeout)
Using.resource(new PointerScope()) { _ =>
      for (epoch <- 1 to num_epochs) {
        model.train()
        var totalLoss = 0l
        // val exampleIter = trainLoader.iterator
        var batchIndex = 0
        print("hhh")  
        for ((inputs, targets) <- trainLoader) {
          // 将数据移到目标设备
          val inputsDevice = inputs.to(device)
          val targetsDevice = targets.to(device)

          // 前向传播
          val outputs = model(inputsDevice.reshape(-1, 28, 28))
          // val loss = criterion(outputs, targetsDevice)

          // 反向传播和优化
          optimizer.zeroGrad()
          // loss.backward()
          optimizer.step()

          // totalLoss = totalLoss + loss.item.asInstanceOf[Long]
          batchIndex = batchIndex + 1
          print(s" hhhhhhhhhh..")  
          // 定期评估
          // if (batchIndex % eval_interval == 0) {
          //   val (evalLoss, accuracy) = evaluate()
          //   println(
          //     s"Epoch: $epoch, Iteration: ${batchIndex}, | Training loss: ${loss.item}%.4f " +
          //       s"Train Loss: ${totalLoss / eval_interval}, Eval Loss: $evalLoss, Accuracy: $accuracy"
          //   )
          //   totalLoss = 0.0f
          //   model.train()
          // }
        }

        // 每个epoch结束后评估
        // val (evalLoss, accuracy) = evaluate()
        // println(s"Epoch $epoch completed. Eval Loss: $evalLoss, Accuracy: $accuracy")
      }
    }

hhh hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhhhh.. hhhhhhhh

In [ ]:
trainLoader

In [9]:
val a = torch.rand(4, 3, 28, 28)
val b=a.numpy[Float32]()
b.printArray()

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


13:34:23,020 |-INFO in ch.qos.logback.classic.LoggerContext[default] - This is logback-classic version 1.4.11
13:34:23,021 |-INFO in ch.qos.logback.classic.util.ContextInitializer@39a879f4 - No custom configurators were discovered as a service.
13:34:23,021 |-INFO in ch.qos.logback.classic.util.ContextInitializer@39a879f4 - Trying to configure with ch.qos.logback.classic.joran.SerializedModelConfigurator
13:34:23,022 |-INFO in ch.qos.logback.classic.util.ContextInitializer@39a879f4 - Constructed configurator of type class ch.qos.logback.classic.joran.SerializedModelConfigurator
13:34:23,023 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback-test.scmo]
13:34:23,023 |-INFO in ch.qos.logback.classic.LoggerContext[default] - Could NOT find resource [logback.scmo]
13:34:23,024 |-INFO in ch.qos.logback.classic.util.ContextInitializer@39a879f4 - ch.qos.logback.classic.joran.SerializedModelConfigurator.configure() call lasted 1 milliseconds. ExecutionSta

SLF4J(I): Connected with provider of type [ch.qos.logback.classic.spi.LogbackServiceProvider]


DType is f4
clasz is float clz is int
NDArray init state: data: 9408, size: 9408 itemSize: 9412 ndim: 4,dtype: float32 shape:[4,3,28,28], nBytes: 37648 
2025-09-24 13:34:23 [scala-interpreter-1] DEBUG torch.numpy.matrix.NDArray$ - NDArray Reshape old size: 9408 , new Size 9408
DType is f4
clasz is float clz is int
NDArray init state: data: 4, size: 9408 itemSize: 9412 ndim: 4,dtype: float32 shape:[4,3,28,28], nBytes: 37648 
[
	[
		[
			[0.2739486, 0.047935963, 0.98192906, 0.08739507, 0.7043926, 0.20078713, 0.95390916, 0.92200756, 0.76140434, 0.375714, 0.2725923, 0.9871756, 0.9861154, 0.36538494, 0.77232295, 0.53309447, 0.07809013, 0.85664, 0.5165429, 0.2676726, 0.7303355, 0.64125425, 0.44775122, 0.090981185, 0.1663779, 0.9259948, 0.4121952, 0.76210827],
			[0.15511853, 0.31038576, 0.48066854, 0.77991116, 0.22426349, 0.45480788, 0.12935346, 0.58689636, 0.22143298, 0.7317072, 0.5126224, 0.96301776, 0.17014933, 0.6581576, 0.8271194, 0.55330235, 0.40497673, 0.6484753, 0.49025327, 0.0284134

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



0.3477425, 0.88185346, 0.4520924, 0.065086305, 0.18295217, 0.9456205, 0.5338822, 0.6048958, 0.31229323, 0.4212336, 0.87833035, 0.92712563, 0.18977588, 0.017023206, 0.8156738, 0.7739656, 0.16832572, 0.80304426, 0.4341423, 0.90504175, 0.06763774, 0.47570187, 0.2605378, 0.36703318, 0.585455],
			[0.764631, 0.91504914, 0.73086065, 0.18715966, 0.024721622, 0.9457944, 0.6517513, 0.9608388, 0.24080515, 0.47443593, 0.2820757, 0.20066649, 0.5759264, 0.92741734, 0.43481576, 0.5039507, 0.49745083, 0.13911235, 0.5320244, 0.8938415, 0.7861004, 0.8692167, 0.52922356, 0.9931998, 0.7080528, 0.9954491, 0.7627596, 0.3439216],
			[0.55351967, 0.7979832, 0.2612242, 0.7924945, 0.43338525, 0.16820693, 0.18988872, 0.8641893, 0.041710377, 0.26862782, 0.2504083, 0.053388774, 0.13728625, 0.64108276, 0.41548944, 0.9642899, 0.7410541, 0.76869524, 0.49763227, 0.4863888, 0.9900484, 0.67142487, 0.33885115, 0.6151362, 0.7742754, 0.87620944, 0.8362944, 0.18490118],
			[0.0073627234, 0.8591134, 0.6516232, 0.7742425, 0.

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
val ab = torch.rand(Seq(4, 3, 28, 28))
val aa = ab.index_select(0, torch.tensor(Seq(0, 2))).shape
println(s"aa shape ${aa} , torch.Size([2, 3, 28, 28])")

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


aa shape ArraySeq(2, 3, 28, 28) , torch.Size([2, 3, 28, 28])


ab: Tensor[Float32] = tensor dtype=float32, shape=[4, 3, 28, 28], device=CPU 
[[[[0.5846, 0.0863, 0.0813, ..., 0.8746, 0.1897, 0.9887],
   [0.4520, 0.1207, 0.0747, ..., 0.1701, 0.3551, 0.7436],
   [0.1766, 0.5820, 0.6947, ..., 0.1328, 0.4225, 0.9685],
   ...,
   [0.6126, 0.9037, 0.7590, ..., 0.6845, 0.3120, 0.4158],
   [0.3628, 0.5638, 0.1348, ..., 0.3514, 0.9487, 0.5192],
   [0.1815, 0.8411, 0.3245, ..., 0.8939, 0.8432, 0.6110]],

  [[0.4004, 0.3478, 0.4458, ..., 0.3372, 0.4444, 0.7663],
   [0.0582, 0.9196, 0.7618, ..., 0.3336, 0.6205, 0.5060],
   [0.0971, 0.1336, 0.9332, ..., 0.3513, 0.0486, 0.6753],
   ...,
   [0.6766, 0.6968, 0.5827, ..., 0.6322, 0.6237, 0.8715],
   [0.4516, 0.8600, 0.6768, ..., 0.9607, 0.0142, 0.5553],
   [0.7228, 0.1336, 0.5283, ..., 0.3378, 0.0888, 0.3466]],

  [[0.3048, 0.4438, 0.6177, ..., 0.3962, 0.1869, 0.9107],
   [0.3698, 0.4703, 0.3112, ..., 0.6554, 0.2822, 0.6461],
   [0.0888, 0.3882, 0.7527, ..., 0.1143, 0.9997, 0.2316],
   ...,
   [0.3906, 0.9520, 0.21

In [11]:
val tensor = torch.arange(0, 16).reshape(4, 4)

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.


tensor: Tensor[Int32] = tensor dtype=int32, shape=[4, 4], device=CPU 
[[0, 1, 2, 3],
 [4, 5, 6, 7],
 [8, 9, 10, 11],
 [12, 13, 14, 15]]

In [12]:
val cols = Seq("category", "name", "value","version")
val rows= Seq("row1", "row2", "row3", "row4", "row5", "row6")
val col4Data = Seq("test", "release", "alpha", "beta", "gama", "peter")
val col5Data = Seq("one", "two", "three", "four", "five", "six")
    //  val col6Data = Seq(Float.NaN,Float.NaN,Float.NaN,Float.NaN,Float.NaN,Float.NaN)// Float.NaN, 48, 52, 67)
val col1Data = Seq(14, 25,Float.NaN, 48, 52, 67)
val col2Data = Seq(3, 5, Float.NaN, 0, 9, 10)
val col3Data = Seq(10, 25, 32, 45, 53, 60)
val data = List(col1Data, col2Data, col4Data, col5Data)
    // this(index: KeySet[Any], columns: mutable.Set[Any], data: List[Seq[V]])
var df = new DataFrame(rows, cols, data)
df.show()

cols: Seq[String] = List("category", "name", "value", "version")
rows: Seq[String] = List("row1", "row2", "row3", "row4", "row5", "row6")
col4Data: Seq[String] = List(
  "test",
  "release",
  "alpha",
  "beta",
  "gama",
  "peter"
)
col5Data: Seq[String] = List("one", "two", "three", "four", "five", "six")
col1Data: Seq[Float] = List(14.0F, 25.0F, NaNF, 48.0F, 52.0F, 67.0F)
col2Data: Seq[Float] = List(3.0F, 5.0F, NaNF, 0.0F, 9.0F, 10.0F)
col3Data: Seq[Int] = List(10, 25, 32, 45, 53, 60)
data: List[Seq[scala.collection.immutable.List[scala.collection.immutable.Seq[scala.Float | java.lang.String]]]] = List(
  List(14.0F, 25.0F, NaNF, 48.0F, 52.0F, 67.0F),
  List(3.0F, 5.0F, NaNF, 0.0F, 9.0F, 10.0F),
  List("test", "release", "alpha", "beta", "gama", "peter"),
  List("one", "two", "three", "four", "five", "six")
)
df: DataFrame[torch.pandas.DataFrame[scala.Float | java.lang.String]] = Iterable(
  List(14.0F, 3.0F, "test", "one"),
  List(25.0F, 5.0F, "release", "two"),
  List(NaNF, NaNF, 

In [8]:
df

res8: DataFrame[torch.pandas.DataFrame[scala.Float | java.lang.String]] = Iterable(
  List(14.0F, 3.0F, "test", "one"),
  List(25.0F, 5.0F, "release", "two"),
  List(NaNF, NaNF, "alpha", "three"),
  List(48.0F, 0.0F, "beta", "four"),
  List(52.0F, 9.0F, "gama", "five"),
  List(67.0F, 10.0F, "peter", "six")
)

Using device: Device(CPU,-1)


device: Device = Device(device = CPU, index = -1)
modelPahth: String = "D:\\data\\git\\storch-tutorial\\lstm-netss.pt"

Using device: Device(CPU,-1)


device: Device = Device(device = CPU, index = -1)
modelPahth: String = "D:\\data\\git\\storch-tutorial\\lstm-netss.pt"
dataPath: Path = D:\data\FashionMNIST
mnistTrain: FashionMNIST = IndexedSeq(
  (
    tensor dtype=float32, shape=[1, 28, 28], device=CPU 
[[[0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  ...,
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000]]],
    tensor dtype=int64, shape=[], device=CPU 
9
  ),
  (
    tensor dtype=float32, shape=[1, 28, 28], device=CPU 
[[[0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  ...,
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
  [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0

In [7]:

val batchSize = 100
val timeout = 10.0f
val sampler = new RandomSampler(mnistTrain.length)
val batchSampler = new RandomSampler(mnistTrain.length)
val trainLoader = new DataLoader(dataset = mnistTrain, batch_size= batchSize, shuffle = true, sampler = sampler, batch_sampler= batchSampler, timeout = timeout)
val evalLoader = new DataLoader(dataset = mnistEval,batch_size= batchSize, shuffle = false, sampler = sampler, batch_sampler= batchSampler, timeout = timeout)

batchSize: Int = 100
timeout: Float = 10.0F
sampler: RandomSampler = torch.utils.data.sampler.RandomSampler[address=0x24e8aa5a530,position=0,limit=1,capacity=1,deallocator=org.bytedeco.javacpp.Pointer$NativeDeallocator[ownerAddress=0x24e8aa5a530,deallocatorAddress=0x7ffa6b87f390]]
batchSampler: RandomSampler = torch.utils.data.sampler.RandomSampler[address=0x24e8aa58a10,position=0,limit=1,capacity=1,deallocator=org.bytedeco.javacpp.Pointer$NativeDeallocator[ownerAddress=0x24e8aa58a10,deallocatorAddress=0x7ffa6b87f390]]
trainLoader: DataLoader[Float32] = Iterable(
  (
    tensor dtype=float32, shape=[100, 1, 28, 28], device=CPU 
[[[[0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
   [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
   [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
   ...,
   [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
   [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000],
   [0.0000, 0.0000, 0.0000, ..., 0.0000, 0.0000, 0.0000]]],

 [[[0

In [ ]:
val iter = trainLoader.iterator
iter

In [ ]:

Using.resource(new PointerScope()) { _ =>
      for (epoch <- 1 to num_epochs) {
        model.train()
        var totalLoss = 0.0f
        val exampleIter = trainLoader.iterator
        var batchIndex = 0
        print("hhh")  
        for ((inputs, targets) <- trainLoader) {
          // 将数据移到目标设备
          val inputsDevice = inputs.to(device)
          val targetsDevice = targets.to(device)

          // 前向传播
          val outputs = model(inputsDevice.reshape(-1, 28, 28))
          val loss = criterion(outputs, targetsDevice)

          // 反向传播和优化
          optimizer.zeroGrad()
          loss.backward()
          optimizer.step()

          totalLoss = totalLoss + loss.item.asInstanceOf[Float]
          batchIndex = batchIndex + 1
          print(s" hhhhhhhhhh..")  
          // 定期评估
          // if (batchIndex % eval_interval == 0) {
          //   val (evalLoss, accuracy) = evaluate()
          //   println(
          //     s"Epoch: $epoch, Iteration: ${batchIndex}, | Training loss: ${loss.item}%.4f " +
          //       s"Train Loss: ${totalLoss / eval_interval}, Eval Loss: $evalLoss, Accuracy: $accuracy"
          //   )
          //   totalLoss = 0.0f
          //   model.train()
          // }
        }

        // 每个epoch结束后评估
        // val (evalLoss, accuracy) = evaluate()
        // println(s"Epoch $epoch completed. Eval Loss: $evalLoss, Accuracy: $accuracy")
      }
    }

Resolving package/object name conflict in favor of package spire.compat. The object will be inaccessible.
